In [1]:
# basic
import pandas as pd
import talib
import os.path
import math
from datetime import timedelta, datetime

#time
# import datetime as datetime

#manual
from crypto import test
from binance.client import Client
from backtesting import Backtest , Strategy
from backtesting.lib import SignalStrategy , TrailingStrategy , crossover

C:\ProgramData\Anaconda3\lib\site-packages\backtesting\_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    
    return talib.SMA(values,n)
#     return pd.Series(values).rolling(n).mean()

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 65
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [6]:
ETH_1h = get_all_binance('ETHUSDT', '1h')

bt = Backtest(ETH_1h.loc['2020-01-01':'2099-11-27'], SmaCross, cash=10_000, commission=0)
stats = bt.run()
stats

2021-03-30 08:00:00 2021-03-30 01:56:43.195882
All caught up..!


Start                     2020-01-01 00:00:00
End                       2021-03-30 08:00:00
Duration                    454 days 08:00:00
Exposure Time [%]                     98.3006
Equity Final [$]                      75154.4
Equity Peak [$]                       84381.3
Return [%]                            651.544
Buy & Hold Return [%]                 1305.91
Return (Ann.) [%]                       404.3
Volatility (Ann.) [%]                 557.335
Sharpe Ratio                         0.725415
Sortino Ratio                         8.34278
Calmar Ratio                          11.3704
Max. Drawdown [%]                    -35.5571
Avg. Drawdown [%]                     -4.5577
Max. Drawdown Duration      127 days 10:00:00
Avg. Drawdown Duration        4 days 06:00:00
# Trades                                  316
Win Rate [%]                          31.0127
Best Trade [%]                        48.9796
Worst Trade [%]                      -9.57556
Avg. Trade [%]                    

In [7]:
bt.plot()


C:\ProgramData\Anaconda3\lib\site-packages\backtesting\_plotting.py:104: UserWarning: Data contains too many candlesticks to plot; downsampling to '2H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
